Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import re
import cv2
import json
from pprint import pprint
import random

Helper Functions

In [ ]:
def extract_id(file_name: str) -> str | None:
    """
    Give a file name such as 'A_P000001_PAS_CPG.tif',
    Extract the file ID: 'A_P000001' using regular expression
    """
    match = re.match(r"([A-Z]_P\d+)_", file_name, re.IGNORECASE)

    if match:
        return match.group(1)
    else:
        return None


def cell_mask_to_rgb(cell_mask: np.ndarray) -> np.ndarray:
    """For visualization purposes
    Convert 2D cell mask to an RGB image
    2D cell mask:
    1 - Lympchoyte centroids
    2 - Monocytes centroids
    RGB image:
    Green - Lympchoyte centroids
    Blue - Monocytes centroids
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (13, 13))
    cell_mask = cv2.dilate(cell_mask, kernel, iterations=1)
    rgb_mask = np.zeros(
        (cell_mask.shape[0], cell_mask.shape[1], 3), dtype=np.uint8
    )
    rgb_mask[cell_mask == 1] = (0, 255, 0)
    rgb_mask[cell_mask == 2] = (0, 0, 255)
    return rgb_mask

Visualize 50 random patches

In [ ]:
# Specify data folders

# Folder containing image patches
patch_image_dir = "/home/u1910100/Documents/Monkey/patches_256/images"
# Folder containing cell masks
cell_mask_dir = (
    "/home/u1910100/Documents/Monkey/patches_256/annotations/masks"
)
# Folder containing json files
json_dir = (
    "/home/u1910100/Documents/Monkey/patches_256/annotations/json"
)


# Visualize 50 random patches
file_names = os.listdir(patch_image_dir)
files_sample = random.sample(file_names, 50)

for name in files_sample:
    name_without_ext = os.path.splitext(name)[0]

    # Get path to files
    patch_image_path = os.path.join(
        patch_image_dir, f"{name_without_ext}.npy"
    )
    cell_mask_path = os.path.join(
        cell_mask_dir, f"{name_without_ext}.npy"
    )
    json_path = os.path.join(json_dir, f"{name_without_ext}.json")

    # load RGB patch and cell mask
    image_patch = np.load(patch_image_path)
    cell_mask = np.load(cell_mask_path)
    # Convert cell mask to RGB image for visualization
    rgb_cell_mask = cell_mask_to_rgb(cell_mask)

    # load json file
    annotations = {}
    with open(json_path, "r") as file:
        annotations = json.load(file)

    # Output
    pprint(name)
    # pprint(annotations)
    fig, axes = plt.subplots(1, 3, figsize=(17, 17))
    axes[0].imshow(image_patch)
    axes[0].title.set_text("RGB Patch")
    axes[1].imshow(rgb_cell_mask)
    axes[1].title.set_text("Cell Mask")
    axes[2].imshow(image_patch, alpha=0.5)
    axes[2].imshow(rgb_cell_mask, alpha=0.5)
    axes[2].title.set_text("Cell Mask overlay on RGB Patch")
    plt.show()

Count cells

In [ ]:
# Folder containing json files
json_dir = (
    "/home/u1910100/Documents/Monkey/patches_256/annotations/json"
)
file_names = os.listdir(json_dir)

lymphocyte_count = 0
monocytes_count = 0

for name in file_names:
    json_path = os.path.join(json_dir, name)
    annotations = {}
    with open(json_path, "r") as file:
        annotations = json.load(file)

    lymph_coords = annotations["lymphocytes"]
    mono_coords = annotations["monocytes"]

    lymphocyte_count += len(lymph_coords)
    monocytes_count += len(mono_coords)


print(f"lympho count = {lymphocyte_count}")
print(f"mono count = {monocytes_count}")

In [ ]:
# Create patch stats
json_dir = (
    "/home/u1910100/Documents/Monkey/patches_256/annotations/json"
)

stats_save_path = (
    "/home/u1910100/Documents/Monkey/patches_256/patch_stats.json"
)

patch_stats = {}

p = 0
n = 0

file_names = os.listdir(json_dir)
for name in file_names:
    name_without_ext = os.path.splitext(name)[0]
    json_path = os.path.join(json_dir, name)
    annotations = {}
    with open(json_path, "r") as file:
        annotations = json.load(file)

    lymph_coords = annotations["lymphocytes"]
    mono_coords = annotations["monocytes"]

    patch_stats[name_without_ext] = {
        "lymph_count": len(lymph_coords),
        "mono_count": len(mono_coords),
    }

    if len(lymph_coords) > 0 or len(mono_coords) > 0:
        p += 1
    else:
        n += 1

print(p)
print(n)
with open(stats_save_path, "w") as file:
    json.dump(patch_stats, file)

Visualize Detection Dataloader

In [ ]:
%reset -f
from monkey.data.dataset import get_detection_dataloaders
from monkey.data.data_utils import imagenet_denormalise
from monkey.config import TrainingIOConfig
import numpy as np
import matplotlib.pyplot as plt
from monkey.model.efficientunetb0.architecture import (
    get_efficientunet_b0_MBConv,
)

IOconfig = TrainingIOConfig(
    dataset_dir="/home/u1910100/Documents/Monkey/patches_256",
    save_dir="./",
)
batch_size = 8
train_loader, val_loader = get_detection_dataloaders(
    IOconfig,
    val_fold=1,
    task=1,
    batch_size=batch_size,
    disk_radius=15,
    do_augmentation=False,
    module="detection",
)

In [ ]:
data = next(iter(train_loader))
fig, axes = plt.subplots(batch_size, 3, figsize=(19, 19))

for i in range(batch_size):
    print(data["id"][i])
    image = data["image"][i].numpy()
    mask = data["mask"][i][0].numpy()

    image = np.moveaxis(image, 0, 2)
    # print(np.max(mask))

    image = imagenet_denormalise(image)
    # print(image)
    axes[i][0].imshow(image)
    axes[i][1].imshow(mask, cmap="gray")
    axes[i][2].imshow(image, alpha=0.5)
    axes[i][2].imshow(mask, alpha=0.5)
plt.show()

Visualize Classification DataLoader

In [ ]:
%reset -f
from monkey.data.dataset import get_classification_dataloaders
from monkey.data.data_utils import imagenet_denormalise
from monkey.config import TrainingIOConfig
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import os

IOconfig = TrainingIOConfig(
    dataset_dir="/home/u1910100/Documents/Monkey/classification",
    save_dir="./",
)
IOconfig.set_image_dir(
    "/home/u1910100/Documents/Monkey/classification/patches"
)
IOconfig.set_mask_dir(
    "/home/u1910100/Documents/Monkey/classification/patches"
)
batch_size = 8
train_loader, val_loader = get_classification_dataloaders(
    IOconfig,
    val_fold=1,
    batch_size=batch_size,
    do_augmentation=True,
)

In [ ]:
data = next(iter(val_loader))
fig, axes = plt.subplots(batch_size, 2, figsize=(8, 19))

for i in range(batch_size):
    print(data["id"][i])
    print(data["label"][i])
    image = data["image"][i].numpy()
    mask = data["mask"][i][0].numpy()

    image = np.moveaxis(image, 0, 2)
    image = imagenet_denormalise(image)

    axes[i][0].imshow(image)
    axes[i][1].imshow(mask, cmap="gray")
plt.show()